# Check Whisper ASR notebook for outputs

# USE WSL if in Windows

In [ ]:
clone github repo first

In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

In [ ]:
with open("test.txt", "w") as file:
  file.write("\n".join(list(filter(lambda s: len(s.split()) > 2, swahili_final['train']['transcription']))))

In [ ]:
!kenlm/build/bin/lmplz -o 4 <"test.txt"> "4gram.arpa"

In [ ]:
!head -20 4gram.arpa

In [ ]:
with open("4gram.arpa", "r") as read_file, open("4gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [ ]:
import kenlm
model = kenlm.LanguageModel('kenlm/lm/test.arpa')
print(model.score('M-pesa is a mobile.', bos = True, eos = True))

In [ ]:
import kenlm
model = kenlm.LanguageModel('4gram_correct.arpa')
print(model.score('M-pesa is a mobile.', bos = True, eos = True))

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="kenlm/lm/test.arpa",
)